# Assumptions <br>
1. puser will be used as the PersonHistoricalId <br>
2. imAddresses as the email address <br>
3. PopulationType column assumes all are licensed employees (not true because it will include conference rooms) 
4. All people are assumed to be internal


In [7]:
# Getting inputs for accessing the blob
StorageAccount = "dopsis"
BLOBcontainer = "ona/MGDC_20210817"
RawDataContainer = "users/rawdata/MGDC_data"

# Setting connection
(input_container, raw_data_container) = (RawDataContainer.split("/")[:2])
(out_container, ona_container) = (BLOBcontainer.split("/")[:2])
 
wasbs_template = "abfss://%s@%s.dfs.core.windows.net/%s/{0}/" % (input_container, StorageAccount, raw_data_container)

wasbs_template_out = "abfss://%s@%s.blob.core.windows.net/%s/{0}/" % (out_container, StorageAccount, ona_container)


StatementMeta(spark1, 53, 7, Finished, Available)

In [8]:
from pyspark.sql.functions import to_date, date_format, ceil, year, lit, udf, explode, split, last_day, trunc, monotonically_increasing_id, lower
import os
import pyspark.sql.functions as F
import datetime as dt
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.window import Window

import networkx as nx
import topologic as tc
import graspologic as gc

from datetime import datetime, timedelta
import pandas as pd
from notebookutils import mssparkutils

StatementMeta(spark1, 53, 8, Finished, Available)

In [9]:
def df_output_blob(df, extension, outFolder):
    outPath = ResultBlobPath + outFolder + "/"
    if extension == 'csv':
      df.repartition(1).write.csv(outPath, header='true', mode='overwrite', escape="\"")
    elif extension == 'json':
      df.repartition(1).write.json(outPath)
    
    # Copy file from outFolder to central working directory
    try:
      fullLS = mssparkutils.fs.ls(outPath)
      for i in fullLS:
        if 'part-00000' in i.name:
          outFileName = i.name
          outFileLocation = i.path
          newFileLocation = ResultBlobPath + outFolder + '.' + extension
          mssparkutils.fs.mv(outFileLocation, newFileLocation, overwrite=True)
          print ('File moved successfully: ', newFileLocation)
    except Exception as e:
      print ("Error moving file. Error: ", e) 
  
    # clean up old files
    try:
      mssparkutils.fs.rm(outPath , True)
      print ('Work Folder deleted: ', outPath)
    except Exception as e:
      print ("Error Deleting work File or Folder. Error: ", e)

ResultBlobPath = wasbs_template.format('MGDC_Viva_raw')
print("Raw Data Container: ", ResultBlobPath)

StatementMeta(spark1, 53, 9, Finished, Available)

Raw Data Container:  abfss://users@dopsis.dfs.core.windows.net/rawdata/MGDC_Viva_raw/

In [10]:
# Create the personhistorical file
users = spark.read.option('header', 'true').json(wasbs_template.format('MGDC_users'))
user_select_columns = [col('id').alias('PersonHistoricalId'),
                        'imAddresses',
                        col('createdDateTime').alias('EffectiveDate'),
                        col('department').alias('Organization'),
                        'jobTitle', 
                        col('hireDate').alias('StartDate'), 
                        'companyName', 
                        'ptenant']
csvPersonHistorical = users.select(user_select_columns).withColumn('EmailAddress', F.explode('imAddresses'))
csvPersonHistorical = (csvPersonHistorical  
    .withColumn("Domain", F.substring_index(csvPersonHistorical.EmailAddress, '@', -1))
    .withColumn('PopulationType',lit('MeasuredEmployee'))
    .withColumn('EndDate',lit('9999-12-31T23:59:59.9999999'))
    .withColumn('IsInternal',lit(True))
    .select('PersonHistoricalId','EmailAddress','StartDate','EndDate','PopulationType',
    'Domain','EffectiveDate','jobTitle','companyName','ptenant','Organization','IsInternal'))

print("Total Number of User Records: ", csvPersonHistorical.count())

df_output_blob(csvPersonHistorical,'csv','PersonHistorical')

StatementMeta(spark1, 53, 10, Finished, Available)

Total Number of User Records:  186
File moved successfully:  abfss://users@dopsis.dfs.core.windows.net/rawdata/MGDC_Viva_raw/PersonHistorical.csv
Work Folder deleted:  abfss://users@dopsis.dfs.core.windows.net/rawdata/MGDC_Viva_raw/PersonHistorical/

In [12]:
# Create the Mails and MailsParticipants files
# Mails.csv file
### TO DO ###  Add the Cc and Bcc Recipients to the recipients list
rawMails = spark.read.option("header", "true").json(wasbs_template.format('MGDC_emails'))

Mails = (rawMails
    .select(col("Id").alias("MailId"),"ConversationId","Subject",col("SentDateTime").alias("SentTime"),"ToRecipients.EmailAddress.Address")
    .withColumn("SenderTimeSpentInMinutes",lit(5)))

recipients = (Mails
    .select("MailId","SentTime","Address")
    .withColumn("Recipients", explode(col("Address")))
    .groupBy("MailId","SentTime").agg(F.count("Recipients").alias("NumberofRecipients")))

csvMails = (Mails.select("MailId","ConversationId","Subject","SenderTimeSpentInMinutes","SentTime")
    .join(recipients, on=['MailId','SentTime'], how='left')
    .select("MailId","ConversationId","Subject","SentTime","SenderTimeSpentInMinutes","NumberofRecipients"))


# MailsParticipants.csv file
mp_sender = (rawMails
    .select(col("Id").alias("MailId"),
        col("SentDateTime").alias("LocalSentTime"),
        col("Sender.EmailAddress.Address").alias("EmailAddress"))
    .withColumn("IsSender",lit(True))
    .withColumn("PersonTimeSpentInHours",lit(5/60))
    .withColumn("PersonTimeSpentInMinutes", lit(5)))

mp_receiver = (rawMails
    .select(col("Id").alias("MailId"),
        col("SentDateTime").alias("LocalSentTime"),
        col("ToRecipients.EmailAddress.Address").alias("EmailAddress"))
    .withColumn("EmailAddress", explode(col("EmailAddress")))
    .withColumn("IsSender",lit(False))
    .withColumn("PersonTimeSpentInHours",lit(2.5/60))
    .withColumn("PersonTimeSpentInMinutes", lit(2.5)))

csvMailParticipants = (mp_sender.union(mp_receiver).sort(["LocalSentTime","MailId","IsSender"])
    .join(csvPersonHistorical.select("EmailAddress","PersonHistoricalId"), on="EmailAddress", how='left')
    .select("MailId","PersonHistoricalId","IsSender","LocalSentTime","PersonTimeSpentInHours","PersonTimeSpentInMinutes")
    .where(col('PersonHistoricalId').isNotNull()))

print("Mail count: ", csvMails.count())
print("Mail Participants count: ", csvMailParticipants.count())

df_output_blob(csvMails,'csv','Mails')
df_output_blob(csvMailParticipants,'csv','MailParticipants')

StatementMeta(spark1, 53, 12, Finished, Available)

Mail count:  1073335
Mail Participants count:  2978015
File moved successfully:  abfss://users@dopsis.dfs.core.windows.net/rawdata/MGDC_Viva_raw/Mails.csv
Work Folder deleted:  abfss://users@dopsis.dfs.core.windows.net/rawdata/MGDC_Viva_raw/Mails/
File moved successfully:  abfss://users@dopsis.dfs.core.windows.net/rawdata/MGDC_Viva_raw/MailParticipants.csv
Work Folder deleted:  abfss://users@dopsis.dfs.core.windows.net/rawdata/MGDC_Viva_raw/MailParticipants/

In [13]:
# Create the Meetings and MeetingsParticipants files
# Meetings.csv file
### TO DO ###  Add the Cc and Bcc Recipients to the recipients list
rawMeetings = spark.read.option("header", "true").json(wasbs_template.format('MGDC_meetings'))

Meetings = (rawMeetings
    .select(col("Id").alias("MeetingId"),
            col("iCalUId").alias("ICalUid"),
            col("subject").alias("Subject"),
            col("recurrence").alias("IsRecurring"),
            col("isCancelled").alias("IsCancelled"),
            F.to_timestamp(col("start.dateTime")).alias("LocalStartTime"),
            F.unix_timestamp(F.to_timestamp(col("start.dateTime"))).alias("StartTime"),
            F.unix_timestamp(F.to_timestamp(col("end.dateTime"))).alias("EndTime"),
            col("attendees"))
    .withColumn("IsRecurring",col("IsRecurring").isNotNull())
    .withColumn("DurationMinutes", (col("EndTime") - col("StartTime"))/60)
    .withColumn("DurationHours", col("DurationMinutes")/60)
    .drop("EndTime"))

attendees = (rawMeetings
    .select(col("Id").alias("MeetingId"),
            col("iCalUId").alias("ICalUid"),
            F.unix_timestamp(F.to_timestamp(col("start.dateTime"))).alias("StartTime"),
            "attendees.emailAddress.address")
    .withColumn("address", explode("address"))
    .groupBy("MeetingId","ICalUid","StartTime").agg(F.count("address").alias("TotalAttendees")))

acceptance = (rawMeetings
    .select(col("Id").alias("MeetingId"),
            col("iCalUId").alias("ICalUid"),
            F.unix_timestamp(F.to_timestamp(col("start.dateTime"))).alias("StartTime"),
            "attendees.status.response")
    .withColumn("response", explode("response"))
    .withColumn("response", F.split(col("response"),"'")[1])
    .groupBy("MeetingId","ICalUid","StartTime").pivot("response").count().fillna(0)
    .withColumnRenamed("accepted","TotalAccept")
    .withColumnRenamed("declined","TotalDecline")
    .withColumn("TotalNoResponse", col("none")+col("notResponded"))
    .drop("notResponded", "none"))

csvMeetings = (Meetings.join(attendees, on=["MeetingId", "ICalUid", "StartTime"], how="left")
                .join(acceptance, on=["MeetingId", "ICalUid", "StartTime"], how='left')
                .select('MeetingId', 'ICalUid', 'Subject', 'IsRecurring', 'IsCancelled',
                col('LocalStartTime').alias("StartTime"), 'DurationHours','DurationMinutes', 'TotalAccept', 'TotalDecline', 'TotalNoResponse',
                'TotalAttendees'))


# MeetingParticipants.csv file
csvMeetingParticipants = (rawMeetings
    .select(col("Id").alias("MeetingId"),
            col("attendees"),
            F.to_timestamp(col("start.dateTime")).alias("UTCStartTime"),
            F.unix_timestamp(F.to_timestamp(col("start.dateTime"))).alias("StartTime"),
            F.unix_timestamp(F.to_timestamp(col("end.dateTime"))).alias("EndTime"),
            col('organizer.emailAddress.address').alias('Organizer'))
    .withColumn("DurationMinutesAdjusted", (col("EndTime") - col("StartTime"))/60)
    .withColumn("new", F.arrays_zip("attendees.emailAddress.address", "attendees.status.response"))
    .withColumn("new", F.explode("new"))
    .withColumn("EmailAddress",col("new.0"))
    .withColumn("Response", F.split(col("new.1"),"'")[1])
    .withColumn("IsOrganizer",col("EmailAddress")==col("Organizer"))
    .join(csvPersonHistorical.select("EmailAddress","PersonHistoricalId"), on="EmailAddress", how='left')
    .select("MeetingId","PersonHistoricalId","UTCStartTime","IsOrganizer","Response","DurationMinutesAdjusted")
    .where(col('PersonHistoricalId').isNotNull()))

print("Meeting count: ", csvMeetings.count())
print("Meeting Participants count: ", csvMeetingParticipants.count())

df_output_blob(csvMeetings,'csv','Meetings')
df_output_blob(csvMeetingParticipants,'csv','MeetingParticipants')

StatementMeta(spark1, 53, 13, Finished, Available)

Meeting count:  221602
Meeting Participants count:  741451
File moved successfully:  abfss://users@dopsis.dfs.core.windows.net/rawdata/MGDC_Viva_raw/Meetings.csv
Work Folder deleted:  abfss://users@dopsis.dfs.core.windows.net/rawdata/MGDC_Viva_raw/Meetings/
File moved successfully:  abfss://users@dopsis.dfs.core.windows.net/rawdata/MGDC_Viva_raw/MeetingParticipants.csv
Work Folder deleted:  abfss://users@dopsis.dfs.core.windows.net/rawdata/MGDC_Viva_raw/MeetingParticipants/